# 02 - Data from the Web

## Part 0 - Helpers and constants

#### Import modules

In [5]:
import requests

from bs4 import BeautifulSoup

import re

import pickle

import pandas as pd

#### Constants

In [6]:
# Base URL for isa.epfl.ch public report searches
isa_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/'

#### Helpers

In [7]:
# Helper for creating search path
def isa_url(path):
    return isa_base_url + path

In [8]:
# Allow accessing dictionary items as object attributes
# https://goodcode.io/articles/python-dict-object/
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)

In [9]:
# Fetch the given url using a GET requests with the supplied parameters
def get_page(url, params=None):
    r = requests.get(url, params)
    
     # Check whether the request succeeded or not
    if r.status_code is not requests.codes.ok:
        print("Something went wrong. Got status code = " + str(r.status_code))
        return None
    
    # TODO: Check Content-Length
    
    # Extract the response body
    html = r.text
    
    # Parse it with BeautifulSoup
    return BeautifulSoup(html, 'html.parser')

## Part 1 - Scraper Module

The first step is to fetch all the data from ISA.

To do this, we will have to scrape the respective frames that ISA serves for the form and then the search results. Steps are detailed below.

In [10]:
# URL of the public ISA home page.
isa_home_url = isa_url('%21gedpublicreports.htm?ww_i_reportmodel=133685247')

#### Get the form URL from the home page

In [11]:
home_soup = get_page(isa_home_url)

In [12]:
# Find the form frame
toc_frame = home_soup.find('frame', attrs={'name': 'toc'})

In [13]:
# Extract the form's URL
toc_frame_url = toc_frame.attrs['src']
toc_frame_url # Sanity check

'!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'

In [14]:
# Repeat previous process for the form's frame
toc_url = isa_url(toc_frame_url)

#### Get the form

In [15]:
toc_soup = get_page(toc_url)

In [16]:
# Find the form itself within the page
# The form's name is a heuristic, read from the page's source
form_elem = toc_soup.find('form', attrs={'name': 'f'})

In [17]:
# Find the form's action on validation
form_action = form_elem['action']
form_action_url = isa_url(form_action)
form_action_url # Debug print

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter'

In [18]:
# Define a payload dictionary with all the parameters expected by ISA's API endpoint

# /!\ By default we will request all possibilities from the endpoint, and then filter. /!\ #

# Example requests for specific academic or pedagogic periods are left FYI.

payload = {
    'ww_b_list': '1',
    'ww_i_reportmodel': '133685247',
    'ww_c_langue': '',
    
    # Format = html
    'ww_i_reportModelXsl': '133685270', 
    
    # Faculty = { Informatique : 249847 }
    'zz_x_UNITE_ACAD': 'Informatique',
    'ww_x_UNITE_ACAD': '249847',
    
    # Academic Period = { '' : null } if we want all possible periods
    #                 = { '2016-2017' : 355925344 } for a specific year
    'zz_x_PERIODE_ACAD': '',
    'ww_x_PERIODE_ACAD': 'null',

    
    # Pedagogic Period = { '' : null } if we want all possible periods
    #                  = { 'Bachelor semestre 1' : 249108 } for a specific period
    'zz_x_PERIODE_PEDAGO': '',
    'ww_x_PERIODE_PEDAGO': 'null',
    
    # Winter or summer semesters
    'zz_x_HIVERETE': '',
    'ww_x_HIVERETE': 'null',
    
    'dummy': 'ok'
}

#### Fill in the form and parse the search results

In [19]:
# Parse the result
form_result_soup = get_page(form_action_url, params=payload)
# form_result_soup.prettify() # Debug print

In [20]:
# Extract the links from the list of results
all_links = form_result_soup('a', class_='ww_x_GPS')
# all_links # Debug print

In [21]:
# Store the links in a dictionary. Some cleaning is applied, such as whitespace stripping
# Note : removing 'Tous' allows us to iterate and filter independently each result
#        instead of an aggregate HTML table which would be much more painful to parse
links = [
    objdict({
        'text': link.text.strip(),
        'url': link.attrs['onclick']
    }) for link in all_links if link.text.strip() != 'Tous'
]

For each link,the page loads the respective results with the following on-click action :

```
loadReport('ww_x_GPS=2021043255');return false;
```

We will use the following function ```parse_url()``` to extract the ```ww_x_GPS``` parameter expected by the API to return the desired information about the students.

In [22]:
parse_url_pattern = r"""'(\w+)=(\d+)'"""
parse_url_re = re.compile(parse_url_pattern)

def parse_url(url):
    match = re.search(parse_url_pattern, url)

    if match == None:
        print('parse_url: parsing failure: %s' % url)
        return None
    
    return objdict({
        'key': match.group(1),
        'value': match.group(2)
    })

The list itself must be parsed to extract information regarding section, academic year, pedagogic period. The format is the following :

```
Informatique, 2016-2017, Bachelor semestre 1
```

We call an element of the list an item, and it will be parsed by the following ```parse_item()``` function.

In [23]:
parse_item_pattern = r"""Informatique,\s*(?P<start_year>\d+)-(?P<end_year>\d+),\s*(?P<degree>[\w]+) semestre (?P<semester>\d)"""
parse_item_re = re.compile(parse_item_pattern)

def parse_item(item, url, payload):
    match = re.match(parse_item_re, item)
    
    # The above regex will match only the kind of records we are interested in, so we can
    # safely drop all the non-matching ones
    if match == None:
        # print('Match failed: %s ' % item)
        return None
    
    parsed_url = parse_url(url)
    
    if parsed_url == None:
        return None
    
    params = payload.copy()
    params[parsed_url.key] = parsed_url.value
    
    return objdict({
        'start_year': int(match.group('start_year')),
        'end_year': int(match.group('end_year')),
        'degree': match.group('degree'),
        'semester': int(match.group('semester')),
        'params': params
    })

# Filter all None elements from the given list
def filter_none(a_list):
    return [item for item in a_list if item != None]

In [24]:
all_items = [parse_item(link.text, link.url, payload) for link in links]

In [25]:
def is_valid_item(item):
    return item != None and item.start_year >= 2007 and item.end_year <= 2017

def filter_items(items):
    return [item for item in items if is_valid_item(item)]

In [26]:
items = filter_items(all_items)
items

[{'degree': 'Bachelor',
  'end_year': 2008,
  'params': {'dummy': 'ok',
   'ww_b_list': '1',
   'ww_c_langue': '',
   'ww_i_reportModelXsl': '133685270',
   'ww_i_reportmodel': '133685247',
   'ww_x_GPS': '71297531',
   'ww_x_HIVERETE': 'null',
   'ww_x_PERIODE_ACAD': 'null',
   'ww_x_PERIODE_PEDAGO': 'null',
   'ww_x_UNITE_ACAD': '249847',
   'zz_x_HIVERETE': '',
   'zz_x_PERIODE_ACAD': '',
   'zz_x_PERIODE_PEDAGO': '',
   'zz_x_UNITE_ACAD': 'Informatique'},
  'semester': 1,
  'start_year': 2007},
 {'degree': 'Bachelor',
  'end_year': 2008,
  'params': {'dummy': 'ok',
   'ww_b_list': '1',
   'ww_c_langue': '',
   'ww_i_reportModelXsl': '133685270',
   'ww_i_reportmodel': '133685247',
   'ww_x_GPS': '71297550',
   'ww_x_HIVERETE': 'null',
   'ww_x_PERIODE_ACAD': 'null',
   'ww_x_PERIODE_PEDAGO': 'null',
   'ww_x_UNITE_ACAD': '249847',
   'zz_x_HIVERETE': '',
   'zz_x_PERIODE_ACAD': '',
   'zz_x_PERIODE_PEDAGO': '',
   'zz_x_UNITE_ACAD': 'Informatique'},
  'semester': 2,
  'start_year':

#### Going through the years to extract the list of enrolled students

In [33]:
list_page_url = isa_url('!GEDPUBLICREPORTS.html')

def parse_student_row(tr, item, headers):
    tds = tr.find_all('td')
    texts = [td.text for td in tds]
    
    res = {}
    for i, header in enumerate(headers):
        res[header] = texts[i]
    
    item_keys = ['degree', 'semester', 'start_year', 'end_year']
    for key in item_keys:
        res[key] = item[key]
    
    return objdict(res)

def parse_students_list(page_soup, item):
    table = page_soup.find('table')
    trs = table.find_all('tr')
    
    if len(trs) < 3:
        return []
    
    (header_row, students_rows) = (trs[1], trs[2:])
        
    headers = [x.text for x in header_row.find_all('th')]
    
    return [
        parse_student_row(student_row, item, headers) for student_row in students_rows
    ]

def load_student_page(item):
    return objdict({
        'soup': get_page(list_page_url, item.params),
        'item': item
    })

def flatten_list(a_list):
    return [item for sublist in a_list for item in sublist]

# TODO: Add logging statements above to see the progress

In [34]:
load_from_disk = True
save_path = "students_data.p"

if load_from_disk:
    student_list = pickle.load(open(save_path, 'rb'))
else:
    students_pages = [load_student_page(item) for item in items]

    students_lists = [
        parse_students_list(page.soup, page.item) for page in students_pages
    ]

    student_list = flatten_list(students_lists)
    
    # Serialize the whole data to disk
    pickle.dump(student_list, open(save_path, 'wb'))

In [35]:
students_data = pd.DataFrame(student_list)

def load_bsc_data(data):
    bsc_data = students_data[['No Sciper', 'Civilité', 'Nom Prénom', 'degree', 'semester', 'start_year', 'end_year']]
    return bsc_data.rename(columns={'No Sciper': 'sciper', 'Civilité': 'title', 'Nom Prénom': 'name'})
    
bsc_data = load_bsc_data(students_data)

In [601]:
def filter_bsc_students(bsc_data):
    bsc_data = bsc_data[bsc_data['degree'] == 'Bachelor']
    idx1 = set(bsc_students[bsc_students['semester'] == 1].set_index('sciper').index)
    idx6 = set(bsc_students[bsc_students['semester'] == 6].set_index('sciper').index)

    idx = idx1.intersection(idx6)

    return bsc_students[bsc_students['sciper'].isin(idx)].set_index('sciper').sort_index()

bsc = filter_bsc_students(bsc_data)
bsc

,title,name,degree,semester,start_year,end_year
sciper,,,,,,
147008,Monsieur,Good Xavier,Bachelor,3,2009,2010
147008,Monsieur,Good Xavier,Bachelor,2,2008,2009
147008,Monsieur,Good Xavier,Bachelor,6,2010,2011
147008,Monsieur,Good Xavier,Bachelor,5,2010,2011
147008,Monsieur,Good Xavier,Bachelor,1,2008,2009
147008,Monsieur,Good Xavier,Bachelor,4,2009,2010
169569,Monsieur,Arévalo Christian,Bachelor,3,2008,2009
169569,Monsieur,Arévalo Christian,Bachelor,4,2008,2009
169569,Monsieur,Arévalo Christian,Bachelor,5,2009,2010


In [602]:
def add_semesters_count(bsc):
    cols = ['sciper', 'semester']
    semester_count = bsc.reset_index()[cols].groupby('sciper').count().rename(columns={'semester': 'semester_count'})
    return bsc.merge(semester_count, left_index=True, right_index=True)
    
with_counts = add_semesters_count(bsc)
with_counts.head(20)

,title,name,degree,semester,start_year,end_year,semester_count
sciper,,,,,,,
147008,Monsieur,Good Xavier,Bachelor,3,2009,2010,6
147008,Monsieur,Good Xavier,Bachelor,2,2008,2009,6
147008,Monsieur,Good Xavier,Bachelor,6,2010,2011,6
147008,Monsieur,Good Xavier,Bachelor,5,2010,2011,6
147008,Monsieur,Good Xavier,Bachelor,1,2008,2009,6
147008,Monsieur,Good Xavier,Bachelor,4,2009,2010,6
169569,Monsieur,Arévalo Christian,Bachelor,3,2008,2009,6
169569,Monsieur,Arévalo Christian,Bachelor,4,2008,2009,6
169569,Monsieur,Arévalo Christian,Bachelor,5,2009,2010,6


In [603]:
abc = with_count[['title', 'name', 'semester_count']].reset_index().groupby('sciper').agg('first')
abc.sample(10)

,title,name,semester_count
sciper,,,
205687,Monsieur,Lau Schut Hang Pascal,8
216702,Monsieur,Rudelle Matthieu François Edgard,6
214632,Monsieur,Duhem Martin Nicolas,6
225776,Monsieur,Kaufmann Joël Laurent,8
212686,Monsieur,Urien Thibault Félix,8
180094,Monsieur,Mirza Elias,6
186598,Monsieur,Hugonot Joachim,8
213664,Monsieur,Bouquet Stéphane,6
203909,Monsieur,Rutz Valentin Bernard Roger Alain,6


In [604]:
by_title = abc.groupby('title')
by_title['semester_count'].describe()

title          
Madame    count     28.000000
          mean       6.642857
          std        1.129218
          min        6.000000
          25%        6.000000
          50%        6.000000
          75%        7.250000
          max       10.000000
Monsieur  count    347.000000
          mean       7.103746
          std        1.507462
          min        4.000000
          25%        6.000000
          50%        6.000000
          75%        8.000000
          max       12.000000
Name: semester_count, dtype: float64

In [605]:
def load_msc_data(data):
    cols = [
        'Civilité', 'No Sciper', 'Nom Prénom', 'Statut',
        'Spécialisation', 'Mineur',
        'degree', 'semester', 'end_year', 'start_year'
    ]
    rename = {
        'Civilité': 'title',
        'No Sciper': 'sciper',
        'Nom Prénom': 'name',
        'Statut': 'status',
        'Spécialisation': 'specialisation',
        'Mineur': 'minor'
    }
    
    return students_data[cols].rename(columns=rename)
    
msc_data = load_msc_data(students_data)

def filter_msc_students(students_data):
    return students_data[(students_data['degree'] == 'Master') & (students_data['status'] == 'Présent')]

msc = filter_msc_students(msc_data).set_index('sciper').sort_index()
msc

,title,name,status,specialisation,minor,degree,semester,end_year,start_year
sciper,,,,,,,,,
128911,Monsieur,Gulati Asheesh,Présent,Internet computing,,Master,3,2008,2007
128911,Monsieur,Gulati Asheesh,Présent,Internet computing,,Master,2,2008,2007
129093,Monsieur,Zhou Maoan,Présent,,,Master,2,2008,2007
129326,Monsieur,Ni Zhong Zhong,Présent,,,Master,3,2008,2007
129326,Monsieur,Ni Zhong Zhong,Présent,,,Master,2,2008,2007
138088,Monsieur,Droz-dit-Busset Arnault,Présent,,,Master,1,2008,2007
145546,Monsieur,Clivaz Jean-Philippe,Présent,,,Master,3,2008,2007
145957,Monsieur,Hügli Michael,Présent,,,Master,3,2008,2007
146330,Monsieur,Cardinaux Damien,Présent,,,Master,2,2008,2007


In [606]:
msc_count = add_semesters_count(msc)
msc_count[(msc_count['semester_count'] < 3) & (msc_count['minor'] != '')]

,title,name,status,specialisation,minor,degree,semester,end_year,start_year,semester_count
sciper,,,,,,,,,,
153642,Monsieur,Imhoff Lionel,Présent,,"Mineur en Management, technologie et entrepren...",Master,3,2008,2007,1
153645,Monsieur,Rosat Damien,Présent,,"Mineur en Management, technologie et entrepren...",Master,3,2008,2007,1
160391,Monsieur,Chipounov Vitaly,Présent,,"Mineur en Management, technologie et entrepren...",Master,3,2008,2007,1
172257,Monsieur,Dubut Frédéric,Présent,,"Mineur en Management, technologie et entrepren...",Master,3,2008,2007,1
199294,Monsieur,Jin Xuedong,Présent,,"Mineur en Management, technologie et entrepren...",Master,2,2011,2010,2
199294,Monsieur,Jin Xuedong,Présent,,"Mineur en Management, technologie et entrepren...",Master,3,2012,2011,2
200419,Monsieur,Jing Lifu,Présent,,"Mineur en Management, technologie et entrepren...",Master,2,2011,2010,2
203301,Monsieur,Nishida Keishi,Présent,,"Mineur en Management, technologie et entrepren...",Master,2,2011,2010,2
203301,Monsieur,Nishida Keishi,Présent,,"Mineur en Management, technologie et entrepren...",Master,1,2011,2010,2


In [607]:
msc_agg = msc_count[['title', 'name', 'minor', 'semester_count']].reset_index().groupby('sciper').agg('first')
msc_agg.sample(10)

,title,name,minor,semester_count
sciper,,,,
268769,Monsieur,Lazarev Nikita,,1
195446,Monsieur,Simond Florian,,4
169795,Monsieur,Scheiben Pascal,,2
255700,Madame,Jain Smriti,,2
210707,Monsieur,Pupykin Kostiantyn,,1
183895,Monsieur,Bonnet Rémi,,2
201600,Monsieur,Grütter Karl Samuel,,4
191447,Monsieur,Dimitrov Anton Rosenov,,2
210637,Monsieur,Hardy Diggory Benjamin,,2


In [613]:
msc_agg[msc_agg['semester_count'] >= 1].groupby('title').describe()

semester_count
title                         
Madame   count      131.000000
         mean         2.503817
         std          1.098069
         min          1.000000
         25%          2.000000
         50%          3.000000
         75%          3.000000
         max          7.000000
Monsieur count      834.000000
         mean         2.482014
         std          1.004330
         min          1.000000
         25%          2.000000
         50%          3.000000
         75%          3.000000
         max          6.000000